In [1]:
from openai import OpenAI
import json

API_KEY = 
client = OpenAI(api_key=API_KEY)

myfile = "Science_japonais.txt"

In [3]:
all_obj = []
cpt = 0
prompt = "Write me an example sentence in Japanese for each of the following 15 words:\n"
f = open(myfile,"r") #anki deck exported as text file

for line in f.readlines():
  cpt +=1
  prompt += line

  if cpt%15==0: #send by group of 15 words to avoid exceeding word capacity

    #Context
    prompt += 'Make sure to write exactly 15 sentences. Each sentence should be related to science. The words are given in the following shape : "TRANSLATIONS" JAPANESE_WORD<br>KANA Present the answer like a json file under the following shape : {"translation" : "TRANSLATIONS", "kanji": JAPANESE_WORD, "kana": KANA, "example": your sentence, "english_translation": translation to the sentence}'

    all_elem = []
    while len(all_elem)!=16: #check there are the right number of sentence
      chat_completion = client.chat.completions.create(
        messages=[
          {
            "role": "user",
            "content": prompt,
          }
        ],
        model="gpt-3.5-turbo",
        )

      response = chat_completion.choices[0].message.content
      while response[0]!="{":
        response = response[1:]
      while response[-1]!="}":
        response = response[:-1]
      all_elem = response.split("{")
      if len(all_elem)!=16:
        print("querrying again")

    for e in all_elem[1:]:
      e = ('{'+e).strip()
      all_obj.append(json.loads(e))

    print("{}".format(cpt))
    print("Number of registered sentences {}".format(len(all_obj)))
    prompt = "Write me an example sentence in Japanese for each of the following 15 words:\n"

#end of the file
prompt += 'Make sure to write exactly 15 sentences. Each sentence should be related to science. The words are given in the following shape : "TRANSLATIONS" JAPANESE_WORD<br>KANA Present the answer like a json file under the following shape : {"translation" : "TRANSLATIONS", "kanji": JAPANESE_WORD, "kana": KANA, "example": your sentence, "english_translation": translation to the sentence}'
chat_completion = client.chat.completions.create(
  messages=[
  {
    "role": "user",
    "content": prompt,
  }
  ],
  model="gpt-3.5-turbo",
)

response = chat_completion.choices[0].message.content
all_elem = response.split("{")
for e in all_elem[1:]:
  e = ('{'+e).strip()
  all_obj.append(json.loads(e))

f.close()


15/410
Register sentences 15
30/410
Register sentences 30
45/410
Register sentences 45
60/410
Register sentences 60
75/410
Register sentences 75
90/410
Register sentences 90
105/410
Register sentences 105


In [4]:
import genanki

#Parameters for Anki deck
with open('styling.css', 'r') as file:
    styling = file.read()

my_model = genanki.Model(
  1977336413,
  'science_jap',
  fields=[
    {'name': 'Kanji'},
    {'name': 'Exemple'},
    {'name': 'Traduction'},
    {'name': 'Kana'},
    {'name': 'Definition'},
  ],
  templates=[
    {
      'name': 'Card 1',
      'qfmt': '{{Kanji}}<br><span class="kana">{{Kana}}</span><br><br><span class="example_sentence">{{Exemple}}</span>',
      'afmt': '{{FrontSide}}<hr id="answer"><span class="defi">{{Definition}}</span><br><span class="example_sentence">{{Traduction}}</span>',
    },
  ],css = styling)

my_deck = genanki.Deck(
  1092237946,
  'science_jap_ex')

In [7]:
#Build anki deck
f = open(myfile,"r")
cpt = 0

#Get exact word in the text file in case it differs from the saved one...
for line in f.readlines():
  infos = line.split("\t") #1: Def, 2: Kanji kana
  infos2 = infos[1].split("<br>")
  if len(infos2)>1:
    kanji = infos2[0]
    kana = infos2[1]
  else:
    kanji = infos[1]
    kana = ""
  eng = infos[0]

  my_note = genanki.Note(
      model=my_model,
      fields=[kanji,all_obj[cpt]["example"],all_obj[cpt]["english_translation"],kana,eng])

  my_deck.add_note(my_note)
  cpt+=1

f.close()

genanki.Package(my_deck).write_to_file('science_jap.apkg')

['Trigonometric ratio', '三角比<br>さんかくひ\n']
['Derivation (in logic)', '導出<br>どうしゅつ\n']
['Differentiation', '微分法<br>びぶんほう\n']
['Background', '背景<br>はいけい\n']
['Relational expression', '関係式<br>かんけいしき\n']
['Development expansion', '発展<br>はってん\n']
['Product-Sum formulae', '和積公式<br>わせきこうしき\n']
['Investigation', '考察<br>こうさつ\n']
['Polynomial', '多項式<br>たこうしき\n']
['Arithmetic sequence', '等差数列<br>とうさすうれつ\n']
['Geometric sequence', '等比数列<br>とうひすうれつ\n']
['Factorization', '因数分解<br>いんすうぶんかい\n']
['Leftside', '左辺<br>さへん\n']
['Rightside', '右辺<br>うへん\n']
['Replace', '置き換える<br>おきかえる\n']
['Identity', '恒等式<br>こうとうしき\n']
['Example', '例題<br>れいだい\n']
['Irreductible', '既約<br>きやく\n']
['ｋ−order Equation', 'ｋ-次式<br>k-じしき\n']
['Factor', '因数<br>いんすう\n']
['n-th root', '乗根<br>じょうこん\n']
['Termes', '項<br>こう\n']
['Sum total', '総和<br>そうわ\n']
['Expression', '表記<br>ひょうき\n']
['Simplify', '簡略化<br>かんりゃくか\n']
['Interpolation', '補間<br>ほかん\n']
['Different and distinct', '相異なる<br>あいことなる\n']
['Rule', '定まり<br>さだまり\n']
['Constant term'